<a href="https://colab.research.google.com/github/Ascendina/codigosPython/blob/master/SerieTemporalAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from google.colab import files
import numpy as np
import time
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm.notebook import tqdm
from pprint import pprint
import itertools
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Importing Data ARIMA

In [ ]:
wb = load_workbook("/content/drive/MyDrive/datasets/RESULTADOS_ARIMA.xlsx")
dataset_name = ["CARSALES","LAKEERIE","GAS","Electricity","PIGS","POLLUTION","REDWINE","SUNSPOT","Nordic","B1H"]

linear_type = "arima"

In [ ]:
#MLP = E, SVR = F, RBF = G

dataset = {}
for name in dataset_name:

    ws = wb[name]
    columns = [ws["A"],ws["B"],ws["C"],ws["D"],ws["E"],ws["F"],ws["G"]]

    sheet = [[] for _ in range(len(columns[0]))]

    for c in columns:
        for index,item in enumerate(c):
            sheet[index].append(item.value)

    sheet = np.array(sheet)
    dataset[name] = {"raw":sheet}

for name,data in dataset.items():
    sheet = dataset[name]["raw"]

    dataset[name]["series"]  = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["arima"]   = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["mlp"]     = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["srv"]     = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["rbf"]     = {"treino":[], "teste":[],"validacao":[]}

    for row in sheet[1:]:
        split_type = row[2]
        if row[4]==None:
            continue
        key=None
        if split_type == "Teste":
            key="teste"
        elif split_type == "Validacao":
            key="validacao"
        elif split_type == "Treinamento":
            key = "treino"
        else:
            continue

        dataset[name]["series"][key].append(float(row[1]))
        dataset[name]["arima"][key] .append(float(row[3]))
        dataset[name]["mlp"][key]   .append(float(row[4]))
        dataset[name]["srv"][key]   .append(float(row[5]))
        dataset[name]["rbf"][key]   .append(float(row[6]))

    for type_data in ["teste","treino","validacao"]:

        dataset[name]["series"][type_data]  =np.array(dataset[name]["series"][type_data])
        dataset[name]["arima"][type_data]   =np.array(dataset[name]["arima"][type_data])
        dataset[name]["mlp"][type_data]     =np.array(dataset[name]["mlp"][type_data])
        dataset[name]["srv"][type_data]     =np.array(dataset[name]["srv"][type_data])
        dataset[name]["rbf"][type_data]     =np.array(dataset[name]["rbf"][type_data])


def extremeVal(reference,val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

fixExtreme = np.vectorize(extremeVal)

for name in dataset.keys():
    reference = np.max(dataset[name]["series"]["treino"])

    for type_data in ["teste","treino","validacao"]:
        dataset[name]["mlp"][type_data] = fixExtreme(reference,dataset[name]["mlp"][type_data])
        dataset[name]["srv"][type_data] = fixExtreme(reference,dataset[name]["srv"][type_data])
        dataset[name]["rbf"][type_data] = fixExtreme(reference,dataset[name]["rbf"][type_data])
        dataset[name]["arima"][type_data] = fixExtreme(reference,dataset[name]["arima"][type_data])

# Importing Data SARIMA

In [ ]:
wb = load_workbook("/content/drive/MyDrive/datasets/RESULTADOS_SARIMA.xlsx")
dataset_name = ["CARSALES","LAKEERIE","GAS","Electricity","PIGS","POLLUTION","REDWINE","SUNSPOT","Nordic","B1H","Airlines"]

linear_type = "sarima"

In [ ]:
#MLP = E, SVR = F, RBF = G

dataset = {}
for name in dataset_name:

    ws = wb[name]
    columns = [ws["A"],ws["B"],ws["C"],ws["D"],ws["E"],ws["F"],ws["G"]]

    sheet = [[] for _ in range(len(columns[0]))]

    for c in columns:
        for index,item in enumerate(c):
            sheet[index].append(item.value)

    sheet = np.array(sheet)
    dataset[name] = {"raw":sheet}

for name,data in dataset.items():
    sheet = dataset[name]["raw"]

    dataset[name]["series"]  = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["sarima"] = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["mlp"]     = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["srv"]     = {"treino":[], "teste":[],"validacao":[]}
    dataset[name]["rbf"]     = {"treino":[], "teste":[],"validacao":[]}

    for row in sheet[1:]:
        split_type = row[1]
        key=None
        if split_type == "TESTE":
            key="teste"
        elif split_type == "VALIDAÇÃO":
            key="validacao"
        elif split_type == "TREINAMENTO":
            key = "treino"
        else:
            continue

        dataset[name]["series"][key].append(float(row[6]))
        dataset[name]["sarima"][key].append(float(row[2]))
        dataset[name]["mlp"][key]   .append(float(row[3]))
        dataset[name]["srv"][key]   .append(float(row[4]))
        dataset[name]["rbf"][key]   .append(float(row[5]))

    for type_data in ["teste","treino","validacao"]:

        dataset[name]["series"][type_data]  =np.array(dataset[name]["series"][type_data])
        dataset[name]["sarima"][type_data]  =np.array(dataset[name]["sarima"][type_data])
        dataset[name]["mlp"][type_data]     =np.array(dataset[name]["mlp"][type_data])
        dataset[name]["srv"][type_data]     =np.array(dataset[name]["srv"][type_data])
        dataset[name]["rbf"][type_data]     =np.array(dataset[name]["rbf"][type_data])


def extremeVal(reference,val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

fixExtreme = np.vectorize(extremeVal)

for name in dataset.keys():
    reference = np.max(dataset[name]["series"]["treino"])

    for type_data in ["teste","treino","validacao"]:
        dataset[name]["mlp"][type_data] = fixExtreme(reference,dataset[name]["mlp"][type_data])
        dataset[name]["srv"][type_data] = fixExtreme(reference,dataset[name]["srv"][type_data])
        dataset[name]["rbf"][type_data] = fixExtreme(reference,dataset[name]["rbf"][type_data])
        dataset[name]["sarima"][type_data] = fixExtreme(reference,dataset[name]["sarima"][type_data])

# General use Functions

In [ ]:
def fitness(name,x,model_name,data_type="treino"):

    window_size = int(len(x)/2)
    weights = x

    size_pred = len(dataset[name]["series"][data_type])
    pred_series = np.zeros(size_pred)

    for w in range(window_size):
        linear      =   np.concatenate([np.zeros(w),dataset[name][linear_type][data_type]]   )
        nonlinear   =   np.concatenate([np.zeros(w),dataset[name][model_name][data_type]])

        linear      = linear    [:size_pred]
        nonlinear   = nonlinear [:size_pred]

        pred_series = np.sum([pred_series,np.multiply(linear,weights[w*2]),np.multiply(nonlinear,weights[w*2 + 1])],axis=0)


    return mse(dataset[name]["series"][data_type],pred_series)

def score(name,model_name,window_size,net,data_type="treino"):

    data = apply_window(window_size,name,data_type=data_type,model_names=model_name)
    x = torch.tensor(np.array([data])).float()
    pred_series = net(x).detach().numpy().squeeze()


    return mse(dataset[name]["series"][data_type],pred_series)

In [ ]:
def apply_window(window_size,name,data_type,model_names=None):

    size_pred = len(dataset[name]["series"][data_type])
    new_data = []

    for w in range(window_size):
        linear = np.concatenate([np.zeros(w),dataset[name][linear_type][data_type]]   )
        linear = linear[:size_pred]
        row = np.array(linear)
        for model_name in model_names:
            nonlinear = np.concatenate([np.zeros(w),dataset[name][model_name][data_type]])
            nonlinear = nonlinear [:size_pred]
            row = np.column_stack([row,nonlinear])

        if len(new_data) == 0:
            new_data = row
        else:
            new_data = np.column_stack([new_data,row])

    return np.array(new_data)

def make_model_name(models):
    name = ""
    for i,n in enumerate(models):
        name += (n)
        if i<(len(models)-1):
            name +="+"
    return name

class Best_Model():
    def __init__(self,criterion,val_loader):
        self.loader = val_loader
        self.criterion = criterion
        self.min_validation_loss = np.inf
        self.__state = None
        self.loss_vec = []

    def update(self,model):
        running_loss = .0

        model.eval()
        with torch.no_grad():
            for x,y in self.loader:
                x = x.to(device)
                y = y.to(device)

                optimizer.zero_grad()

                preds = model(x)
                loss = self.criterion(preds,y)

                running_loss += loss

            val_loss = running_loss/len(self.loader)
            val_loss = val_loss.cpu().detach().numpy()

        self.loss_vec.append(val_loss)

        if val_loss < self.min_validation_loss:
            self.min_validation_loss = val_loss
            self.__state = copy.deepcopy(model.state_dict())

    @property
    def state(self):
        return self.__state

def create_prediction(X,kernel):

    X      = torch.tensor(np.expand_dims(X, axis=(0,1))).float()
    kernel = torch.tensor(np.expand_dims(kernel, axis=(0,1))).float()

    X.to(device)
    kernel.to(device)

    result = F.conv2d(X, kernel, stride=1, padding=0)
    return result.numpy().squeeze().astype("f")

In [ ]:
class Build_Data(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(np.array([X])).float()
        self.y = torch.tensor(np.array([y])).float()
        self.len = self.X.shape[0]
    # Getting the data
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    # Getting length of the data
    def __len__(self):
        return self.len

# Convolutional Model

## With Bias

In [ ]:
class Model(nn.Module):
    def __init__(self,shape_size):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1,1,kernel_size=(1,shape_size),bias=True,stride=1, padding=0)

    def forward(self, x):
        x = self.conv1(x)

        return x

results = {}
models = [["mlp"],["srv"],["rbf"],
          ["mlp","srv"],["mlp","rbf"],["rbf","srv"],
          ["mlp","srv","rbf"]]

In [ ]:

verbose = 0#0,1,2
n_exp = 10
n_it = 1000

pbar = tqdm(list(itertools.product(dataset_name[:],models[:],range(1,10+1))),position=0)

for name,model_name,WINDOW_SIZE in pbar:

            key_code = f"{name}_{make_model_name(model_name)}_W{WINDOW_SIZE}"

            results[key_code] = {}

            data     = apply_window(WINDOW_SIZE,name,data_type="treino",model_names=model_name)
            data_val = apply_window(WINDOW_SIZE,name,data_type="validacao",model_names=model_name)

            series     = np.array([dataset[name]["series"]["treino"]]).reshape(-1,1)
            series_val = np.array([dataset[name]["series"]["validacao"]]).reshape(-1,1)

            dataset_train = Build_Data(data,series)
            dataset_val   = Build_Data(data_val,series_val)

            train_loader = DataLoader(dataset = dataset_train,shuffle=True,batch_size = 10)
            val_loader   = DataLoader(dataset = dataset_val  ,shuffle=True,batch_size = 10)

            if verbose>=1:
                print("-"*100)
                print(f"{WINDOW_SIZE} {name.upper()} {make_model_name(model_name).upper()}")


            local_results = {"train_based":{"train":[],"test":[],"val":[]},"valid_based":{"train":[],"test":[],"val":[]}}


            for exp in range(n_exp):

                pbar.set_description(f"{WINDOW_SIZE} {name.upper()} {make_model_name(model_name).upper()} Experiment = {exp:3d}")

                if verbose>=1:
                    print("_"*50)

                net = Model(data.shape[1])

                optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
                loss_func = torch.nn.MSELoss()

                maxIt = 1000
                model_selector = Best_Model(loss_func,val_loader)
                for it in range(n_it):
                    for x, y in train_loader:

                        prediction = net(x)
                        loss = loss_func(prediction, y)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    model_selector.update(net)
                if verbose>=1:
                    print("")
                if verbose>=2:
                    plt.plot(model_selector.loss_vec)
                    plt.plot(range(maxIt),[model_selector.min_validation_loss for _ in range(maxIt)])
                    plt.show()

                weights = net.conv1.weight.detach().numpy().squeeze()

                rtrain = score(name,model_name,WINDOW_SIZE,net,data_type="treino")
                rval   = score(name,model_name,WINDOW_SIZE,net,data_type="validacao")
                rtest  = score(name,model_name,WINDOW_SIZE,net,data_type="teste")

                if verbose>=1:
                    print("Train Based")
                    print(f"MSE Treino    :{rtrain:.3e}")
                    print(f"MSE Validacao :{rval:.3e}")
                    print(f"MSE Teste     :{rtest:.3e}")

                    print(weights)

                local_results["train_based"]["train"].append(rtrain)
                local_results["train_based"]["test"].append(rval)
                local_results["train_based"]["val"].append(rtest)

                net.load_state_dict(model_selector.state)
                weights = net.conv1.weight.detach().numpy().squeeze()


                rtrain = score(name,model_name,WINDOW_SIZE,net,data_type="treino")
                rval   = score(name,model_name,WINDOW_SIZE,net,data_type="validacao")
                rtest  = score(name,model_name,WINDOW_SIZE,net,data_type="teste")

                if verbose>=1:
                    print("Valid Based")
                    print(f"MSE Treino    :{rtrain:.3e}")
                    print(f"MSE Validacao :{rval:.3e}")
                    print(f"MSE Teste     :{rtest:.3e}")

                    print(weights)

                local_results["valid_based"]["train"].append(rtrain)
                local_results["valid_based"]["test"].append(rval)
                local_results["valid_based"]["val"].append(rtest)

            results[key_code] = local_results


  0%|          | 0/770 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
best = {}
selection = "train_based"
for name in dataset_name:
    best[name] = ("",100000,-1)
    for key in results.keys():
        info = key.split("_")
        val  = np.median(results[key][selection]["val"])
        test = np.median(results[key][selection]["test"])
        if name.upper() == info[0].upper():
            if best[name][1] > val:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",val,test]


print("_"*10 + "Results Train Based" + "_"*10)
for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][2]:.3e}")

In [ ]:
best = {}
selection = "valid_based"
for name in dataset_name:
    best[name] = ("",100000,-1)
    for key in results.keys():
        info = key.split("_")
        val  = np.median(results[key][selection]["val"])
        test = np.median(results[key][selection]["test"])
        if name.upper() == info[0].upper():
            if best[name][1] > val:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",val,test]


print("_"*10 + "Results Valid Based" + "_"*10)
for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][2]:.3e}")

## No Bias

In [ ]:
class Model(nn.Module):
    def __init__(self,shape_size):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1,1,kernel_size=(1,shape_size),bias=False,stride=1, padding=0)

    def forward(self, x):
        x = self.conv1(x)

        return x

results = {}
models = [["mlp"],["srv"],["rbf"],
          ["mlp","srv"],["mlp","rbf"],["rbf","srv"],
          ["mlp","srv","rbf"]]

In [ ]:

verbose = 0#0,1,2
n_exp = 10
n_it = 1000

pbar = tqdm(list(itertools.product(dataset_name[:],models[:],range(1,10+1))),position=0)

for name,model_name,WINDOW_SIZE in pbar:

            key_code = f"{name}_{make_model_name(model_name)}_W{WINDOW_SIZE}"

            results[key_code] = {}

            data     = apply_window(WINDOW_SIZE,name,data_type="treino",model_names=model_name)
            data_val = apply_window(WINDOW_SIZE,name,data_type="validacao",model_names=model_name)

            series     = np.array([dataset[name]["series"]["treino"]]).reshape(-1,1)
            series_val = np.array([dataset[name]["series"]["validacao"]]).reshape(-1,1)

            dataset_train = Build_Data(data,series)
            dataset_val   = Build_Data(data_val,series_val)

            train_loader = DataLoader(dataset = dataset_train,shuffle=True,batch_size = 10)
            val_loader   = DataLoader(dataset = dataset_val  ,shuffle=True,batch_size = 10)

            if verbose>=1:
                print("-"*100)
                print(f"{WINDOW_SIZE} {name.upper()} {make_model_name(model_name).upper()}")


            local_results = {"train_based":{"train":[],"test":[],"val":[]},"valid_based":{"train":[],"test":[],"val":[]}}


            for exp in range(n_exp):

                pbar.set_description(f"{WINDOW_SIZE} {name.upper()} {make_model_name(model_name).upper()} Experiment = {exp:3d}")

                if verbose>=1:
                    print("_"*50)

                net = Model(data.shape[1])

                optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
                loss_func = torch.nn.MSELoss()

                maxIt = 1000
                model_selector = Best_Model(loss_func,val_loader)
                for it in range(n_it):
                    for x, y in train_loader:

                        prediction = net(x)
                        loss = loss_func(prediction, y)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    model_selector.update(net)
                if verbose>=1:
                    print("")
                if verbose>=2:
                    plt.plot(model_selector.loss_vec)
                    plt.plot(range(maxIt),[model_selector.min_validation_loss for _ in range(maxIt)])
                    plt.show()

                weights = net.conv1.weight.detach().numpy().squeeze()

                rtrain = score(name,model_name,WINDOW_SIZE,net,data_type="treino")
                rval   = score(name,model_name,WINDOW_SIZE,net,data_type="validacao")
                rtest  = score(name,model_name,WINDOW_SIZE,net,data_type="teste")

                if verbose>=1:
                    print("Train Based")
                    print(f"MSE Treino    :{rtrain:.3e}")
                    print(f"MSE Validacao :{rval:.3e}")
                    print(f"MSE Teste     :{rtest:.3e}")

                    print(weights)

                local_results["train_based"]["train"].append(rtrain)
                local_results["train_based"]["test"].append(rval)
                local_results["train_based"]["val"].append(rtest)

                net.load_state_dict(model_selector.state)
                weights = net.conv1.weight.detach().numpy().squeeze()


                rtrain = score(name,model_name,WINDOW_SIZE,net,data_type="treino")
                rval   = score(name,model_name,WINDOW_SIZE,net,data_type="validacao")
                rtest  = score(name,model_name,WINDOW_SIZE,net,data_type="teste")

                if verbose>=1:
                    print("Valid Based")
                    print(f"MSE Treino    :{rtrain:.3e}")
                    print(f"MSE Validacao :{rval:.3e}")
                    print(f"MSE Teste     :{rtest:.3e}")

                    print(weights)

                local_results["valid_based"]["train"].append(rtrain)
                local_results["valid_based"]["test"].append(rval)
                local_results["valid_based"]["val"].append(rtest)

            results[key_code] = local_results


  0%|          | 0/770 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
best = {}
selection = "train_based"
for name in dataset_name:
    best[name] = ("",100000,-1)
    for key in results.keys():
        info = key.split("_")
        val  = np.median(results[key][selection]["val"])
        test = np.median(results[key][selection]["test"])
        if name.upper() == info[0].upper():
            if best[name][1] > val:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",val,test]


print("_"*10 + "Results Train Based" + "_"*10)
for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][2]:.3e}")

In [ ]:
best = {}
selection = "valid_based"
for name in dataset_name:
    best[name] = ("",100000,-1)
    for key in results.keys():
        info = key.split("_")
        val  = np.median(results[key][selection]["val"])
        test = np.median(results[key][selection]["test"])
        if name.upper() == info[0].upper():
            if best[name][1] > val:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",val,test]


print("_"*10 + "Results Valid Based" + "_"*10)
for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][2]:.3e}")

# Genetic Algorithm

In [ ]:
def pick(vector):
    luckyNumber = np.random.choice(len(vector))
    return vector[luckyNumber]

def fitness(X,kernel,data_type="treino"):

    pred_series = create_prediction(X,kernel.reshape(1,-1))

    return mse(dataset[name]["series"][data_type],pred_series)


def fitness2(data_treino,data_val,kernel):

    mse_treino = fitness(data_treino,kernel,data_type="treino")
    mse_val    = fitness(data_val,kernel,data_type="validacao")

    return mse_treino,mse_val

In [ ]:
N = 100

N_child = int(N*1.2)
p_cross = 0.9
p_mutation = 0.25
p_local_search = 0.25
MINIMUM = 0.1
t_ind = 1/np.power(2*N,1/4)
t_global = 1/np.power(2*N,1/2)

class ind:
    def __init__(self,kernel_size,func=None,generate_new=True):

        self.func = func
        self.name = name
        self.kernel_size = kernel_size

        if generate_new:
            self.gene = np.random.random(kernel_size)*10 -5
            self.sigma = [2.0 for _ in range(kernel_size)]

            self.update_fitness()

    def update_fitness(self):
        self.fitness = self.func(self.gene)

    def mutate(self):
        choice = np.random.random()
        if choice < p_mutation:
            t_ind = 1/np.power(2*N,1/4)
            t_global = 1/np.power(2*N,1/2)

            global_learning = t_global*np.random.normal(0,1)
            sigma = [self.sigma[i]*np.exp(global_learning +t_ind*np.random.normal(0,1)) for i in range(self.kernel_size)]

            for i in range(len(sigma)):
                if sigma[i]<MINIMUM:
                    sigma[i]=MINIMUM

            self.sigma = sigma

            for i in range(self.kernel_size):
                self.gene[i] +=np.random.normal(0,self.sigma[i])

            self.update_fitness()

        return self


    def local_search(self,X):

        choice = np.random.random()

        if p_local_search < choice:

            diff = np.sum(create_prediction(X,self.gene.reshape(1,-1))) - np.sum(dataset[self.name]["series"]["treino"])
            step = 0.1
            weights = self.gene.copy()

            if diff > 0:
                step *= -1

            for i in range(len(weights)):
                weights[i] += step
                new_fit = self.func(weights)
                if new_fit < self.fitness:
                    self.gene = weights
                    self.fitness = new_fit
                    return self
                else:
                    weights = self.gene.copy()

        return self

    def copy(self):
        child = ind(kernel_size=self.kernel_size,func=self.func,generate_new=False)

        child.gene=self.gene.copy()
        child.sigma=self.sigma.copy()
        child.fitness = self.fitness

        return child

    def crossover(self,partner):

        child = ind(kernel_size=self.kernel_size,func=self.func,generate_new=False)

        child.gene = np.array([(self.gene[i]+partner.gene[i])/2 for i in range(self.kernel_size)])
        child.sigma = [(self.sigma[i]+partner.sigma[i])/2 for i in range(self.kernel_size)]

        child.update_fitness()

        return child

In [ ]:
def make_model_name(models):
    name = ""
    for i,n in enumerate(models):
        name += (n)
        if i<(len(models)-1):
            name +="+"
    return name


def run(name,model,window_size,N=N,N_child=N_child,maxIt=50,step_size=10,verbose=True,save_n = 10):

    if verbose:
        print("="*100)
    if maxIt>500:
        maxIt=500
    model_name = make_model_name(model)

    X_train = apply_window(window_size,name,"treino"    ,model_names=model)
    X_val   = apply_window(window_size,name,"validacao" ,model_names=model)
    X_test =  apply_window(window_size,name,"teste"     ,model_names=model)

    fitness_func = lambda kernel : fitness2(X_train,X_val,kernel)

    pop = [ind(kernel_size=X_train.shape[1],func=fitness_func) for _ in range(N)]

    pop.sort(key=lambda ind: ind.fitness)

    children = []

    gen = 0

    solutions = []

    while True:

        for i in range(save_n):
            solutions.append(pop[i].copy())

        if(gen>=maxIt):
            break

        if verbose:
            if ((gen+1)%int(maxIt/step_size)==0 or gen==0):
                print(f"GEN {gen+1:5d} | Best Fit {pop[0].fitness[0]:.2e} | Avg fit {np.mean([p.fitness[0] for p in pop]):.2e}")

        children = []
        children.append(pop[0].copy())

        for _ in range(N_child):
            choice = np.random.random()
            luckyOne = pick(pop)
            if(choice<p_cross):
                temp_child = luckyOne.crossover(pick(pop))
                temp_child = temp_child.mutate()
                children.append(temp_child.local_search(X_train))

            else:
                temp_child = luckyOne.mutate()
                children.append(luckyOne.local_search(X_train))

        children.sort(key=lambda ind: ind.fitness)
        pop = children[:N].copy()
        gen+=1

    solutions.sort(key=lambda ind: ind.fitness)
    sol_train = solutions[0].gene
    solutions.sort(key=lambda ind: (ind.fitness[1],ind.fitness[0]))
    sol_val = solutions[0].gene

    if verbose:
        print("-"*100)
        print(f"Configuration: Pop Size = {N:5d} Prole Size = {N_child:5d}")
        print(f"Train Solution = {sol_train} | MSE = {fitness(X_test,sol_train,data_type='teste'):.2e}")
        print(f"Val   Solution = {sol_val  } | MSE = {fitness(X_test,sol_val,data_type='teste'):.2e}")
        print("-"*100)
        print("")


    return sol_train,fitness(X_test,sol_train,data_type='teste'),sol_val,fitness(X_test,sol_val,data_type='teste')


In [ ]:
np.random.seed(int(time.time()))

results = {}

models = [["mlp"],["srv"],["rbf"],
          ["mlp","srv"],["mlp","rbf"],["rbf","srv"],
          ["mlp","srv","rbf"]]

pbar = tqdm(list(itertools.product(dataset_name[:],models[:],range(1,10+1))),position=0)
for name,model,window_size in pbar:

    model_name = make_model_name(model)
    key_code = f"{name}_{model_name}_W{window_size}"
    results[key_code] = []
    for exp in range(1):
        pbar.set_description(f"{window_size} {name.upper()} {model_name.upper()} Experiment = {exp:3d}")
        sol_train,mse1,sol_val,mse2 = run(name,
                                                model=model,
                                                window_size=window_size,
                                                verbose=False,
                                                maxIt=500,
                                                step_size=10,
                                                save_n=10)

        results[key_code].append(np.array([mse1,mse2]))
        results[key_code] = np.array(results[key_code])

In [ ]:
print("Results by Train Solution")
best = {}
for name in dataset_name:
    best[name] = ("",100000)
    for key in results.keys():

        info = key.split("_")
        sol_mse = np.median(results[key][:,0])
        if name.upper() == info[0].upper():
            if best[name][1] > sol_mse:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",sol_mse]

for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][1]:.3e}")

In [ ]:
print("Results by Validation Solution")
best = {}
for name in dataset_name:
    best[name] = ("",100000)
    for key in results.keys():

        info = key.split("_")
        sol_mse = np.median(results[key][:,1])
        if name.upper() == info[0].upper():
            if best[name][1] > sol_mse:
                best[name] = [f"{int(info[2][1:]):3d} {info[1]}",sol_mse]

for key in best.keys():
    print(f"{key.upper()}{'-'*(30-len(key.upper()))}\n{best[key][0].upper()}: {best[key][1]:.3e}")